# indexing the rhyme

In [1]:
import os
import sys
import logging
PRJ_BASE ='/mnt/d/personal_working/bavandangvanba'
base_path = PRJ_BASE
sys.path.append(base_path)
from core.utils import (
    vn_grammar_handler,
    db_handler,
    init_logger)
init_logger()

/mnt/d/personal_working/bavandangvanba/


In [2]:
import pandas as pd
n_gram_phrases_song = pd.read_pickle(os.path.join(PRJ_BASE,'tmp/dictionary_from_songs_data.pkl'))
n_gram_phrases_truyen = pd.read_pickle(os.path.join(PRJ_BASE,'tmp/dictionary_from_truyen_data.pkl'))
n_gram_phrases_tho = pd.read_pickle(os.path.join(PRJ_BASE,'tmp/dictionary_from_thivien_data.pkl'))
n_gram_phrases = n_gram_phrases_truyen
for key,count in n_gram_phrases_song.items():
    if key not in n_gram_phrases:
        n_gram_phrases[key] = count*10
    else:
        n_gram_phrases[key] += count*10
for key,count in n_gram_phrases_tho.items():
    if key not in n_gram_phrases:
        n_gram_phrases[key] = count*10
    else:
        n_gram_phrases[key] += count*10
        
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
pho_vocab = tokenizer.get_vocab()
for key,count in pho_vocab.items():
    real_count = 55000 // (count + 1)
    if real_count<1:
        continue
    if key not in n_gram_phrases:
        n_gram_phrases[key] = real_count
    else:
        n_gram_phrases[key] += real_count

/tmp/ipykernel_3952/4042680821.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/giangtran/miniconda3/envs/sd_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
len(n_gram_phrases)

11355412

In [3]:
len(n_gram_phrases)

14817222

In [4]:
from core.utils import tu_handler
tu_handler
tu_handler.bien_the_van

{'ac': ['ăc', 'at'],
 'au': ['ao'],
 'ao': ['au'],
 'ăc': ['ac', 'ăt'],
 'at': ['ac', 'ăt'],
 'ăt': ['ăc', 'at'],
 'êt': ['êc'],
 'êc': ['êt'],
 'ât': ['âc'],
 'an': ['ang'],
 'ang': ['an']}

In [5]:
import itertools
wrong_data = []
van_indexing = {} 
for idx,(word, count) in enumerate(n_gram_phrases.items()):
    cleaned_text = vn_grammar_handler.bo_dau_va_chuyen_van(word)
    matches_01 = vn_grammar_handler.lay_van_cua_tu_hoac_doan(cleaned_text)
    matches_02 = vn_grammar_handler.clean_phu_am(cleaned_text)
    if matches_01!=matches_02:
        wrong_data.append(['something wrong', word, matches_01, matches_02])
    if matches_01 not in van_indexing:
        van_indexing[matches_01] = []
    tmp_vans = matches_01.split()
    if len(tmp_vans)==2:
        if tmp_vans[-1] not in van_indexing:
            van_indexing[tmp_vans[-1]] = []
        van_indexing[tmp_vans[-1]].append((word,count))
    elif len(tmp_vans)==1:
        continue
    tmp_bienthe = []
    for i_van in tmp_vans:
        tmp_bienthe.append([i_van])
        tmp_bienthe[-1].extend(tu_handler.bien_the_van.get(i_van,[]))

    van_combinations = list(itertools.product(*tmp_bienthe))
    for van_com in van_combinations:
        tmp_key = " ".join(van_com)
        if tmp_key not in van_indexing:
            van_indexing[tmp_key] = []
        van_indexing[tmp_key].append((word,count))

for van in van_indexing:
    van_indexing[van].sort(key=lambda x:x[1],reverse=True)
del van_indexing[""]


In [6]:
len(van_indexing)

8311285

In [7]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(os.path.join(PRJ_BASE,'app_metadata/example.db'))

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create a table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS van_indexing
                  (id TEXT PRIMARY KEY, words TEXT)''')

In [8]:
for idx,(van, text) in enumerate(van_indexing.items()):
    text_list = "\n".join(x[0] for x in text)
   # Add data to the table
    data = ('1', 'Hello, world!')
    cursor.execute('INSERT INTO van_indexing (id, words) VALUES (?, ?)', (van,text_list))
    if idx % 10000 == 0:
        logging.info(f'  - processed {idx} van')

2024-03-03 12:56:42 INFO       - processed 0 van
2024-03-03 12:56:44 INFO       - processed 10000 van
2024-03-03 12:56:45 INFO       - processed 20000 van
2024-03-03 12:56:45 INFO       - processed 30000 van
2024-03-03 12:56:46 INFO       - processed 40000 van
2024-03-03 12:56:46 INFO       - processed 50000 van
2024-03-03 12:56:46 INFO       - processed 60000 van
2024-03-03 12:56:46 INFO       - processed 70000 van
2024-03-03 12:56:47 INFO       - processed 80000 van
2024-03-03 12:56:47 INFO       - processed 90000 van
2024-03-03 12:56:48 INFO       - processed 100000 van
2024-03-03 12:56:48 INFO       - processed 110000 van
2024-03-03 12:56:49 INFO       - processed 120000 van
2024-03-03 12:56:49 INFO       - processed 130000 van
2024-03-03 12:56:50 INFO       - processed 140000 van
2024-03-03 12:56:51 INFO       - processed 150000 van
2024-03-03 12:56:51 INFO       - processed 160000 van
2024-03-03 12:56:52 INFO       - processed 170000 van
2024-03-03 12:56:53 INFO       - processed

In [9]:
conn.commit()

# Close the connection
conn.close()

In [11]:
import subprocess

source_file_path = "/mnt/d/personal_working/bavandangvanba/app_metadata/example.db"
destination_file_path = "root@103.146.23.89:/home/root/working/bavandangvanba/app_metadata/example.db"
key_file_path = "/home/giangtran/.ssh/giang_key.pem"

subprocess.run(["scp", "-i", key_file_path, source_file_path, destination_file_path])

KeyboardInterrupt: 

# test old db

In [5]:
import os
import sys
import logging
import pandas as pd
PRJ_BASE ='/mnt/d/personal_working/bavandangvanba'
base_path = PRJ_BASE
sys.path.append(base_path)
from core.utils import (
    vn_grammar_handler,
    db_handler,
    init_logger)
init_logger()

/tmp/ipykernel_21875/847704826.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [8]:
n_gram_phrases_tho = pd.read_pickle(os.path.join(PRJ_BASE,'tmp/dictionary_from_thivien_data.pkl'))
wrong_data = []
van_indexing = {} 
for idx,(word, count) in enumerate(n_gram_phrases_tho.items()):
    cleaned_text = vn_grammar_handler.bo_dau_va_chuyen_van(word)
    matches_01 = vn_grammar_handler.lay_van_cua_tu_hoac_doan(cleaned_text)
    matches_02 = vn_grammar_handler.clean_phu_am(cleaned_text)
    if matches_01!=matches_02:
        wrong_data.append(['something wrong', word, matches_01, matches_02])
    if matches_01 not in van_indexing:
        van_indexing[matches_01] = []
    van_indexing[matches_01].append((word,count))
for van in van_indexing:
    van_indexing[van].sort(key=lambda x:x[1],reverse=True)
del van_indexing[""]


In [12]:
for x in van_indexing.keys():
    print((x, van_indexing[x]))
    break

('ưa', [('mưa', 10762), ('giữa', 9982), ('chưa', 9227), ('xưa', 6424), ('lửa', 5161), ('vừa', 4846), ('nữa', 4404), ('đưa', 3792), ('cửa', 3648), ('nửa', 2784), ('đứa', 2519), ('ngựa', 1913), ('trưa', 1674), ('thưa', 1347), ('tựa', 1344), ('bữa', 1189), ('thừa', 809), ('sữa', 664), ('dừa', 553), ('chửa', 546), ('hứa', 457), ('rửa', 416), ('nhựa', 377), ('lừa', 374), ('ngửa', 369), ('ứa', 344), ('chứa', 308), ('dựa', 295), ('dưa', 293), ('ưa', 278), ('lựa', 244), ('sửa', 240), ('chừa', 206), ('lứa', 186), ('rứa', 160), ('cựa', 158), ('chữa', 152), ('nứa', 149), ('cứa', 145), ('bừa', 118), ('thửa', 106), ('cưa', 97), ('rữa', 96), ('dứa', 71), ('vữa', 48), ('ngứa', 46), ('lưa', 43), ('mựa', 39), ('sưa', 32), ('mửa', 26), ('bửa', 26), ('nưa', 26), ('sứa', 25), ('rựa', 24), ('tứa', 23), ('ngừa', 23), ('xửa', 23), ('lữa', 21), ('bưa', 20), ('bứa', 11), ('tưa', 11), ('vựa', 10), ('khứa', 9), ('gữa', 7), ('đừa', 6), ('ngữa', 5), ('phứa', 4), ('giưa', 3), ('giưã', 3), ('mữa', 3), ('ngưa', 3), (

In [13]:
import sqlite3
# conn = sqlite3.connect(os.path.join(PRJ_BASE,'app_metadata/example_bk_12M.db'))
conn = sqlite3.connect(os.path.join(PRJ_BASE,'app_metadata/example.db'))
cursor = conn.cursor()
#query all rows
for van, words in van_indexing.items():
    cursor.execute(f"SELECT words FROM van_indexing WHERE id = '{van}'")
    rows = cursor.fetchall()
    
    update_query = "UPDATE van_indexing SET words = %s WHERE id = %s"
    data_tuple = (words, van)
    cursor.execute(update_query, data_tuple)
    # update db
    
    
    
    break


In [18]:
rows[0]

('mưa\nchưa\nxưa\nnữa\ngiữa\nđưa\nhứa\nvừa\ntựa\nđứa\ncửa\nlửa\nthưa\nnửa\ntrưa\nbữa\ndừa\nsữa\nthừa\nlừa\ndựa\nngựa\nchứa\nchữa\nrửa\ndưa\ncứa\nưa\nsửa\ncưa\nlứa\nứa\nchừa\nlựa\nrứa\nnhựa\nChưa\nngửa\nthửa\nlưa\ncựa\ngiửa\nsưa\nVừa\ndứa\nngứa\nkhứa\nGiữa\nvựa\nbửa\nxưạ\nCửa\nbừa\ntừa\nĐưa\nBữa\nDựa\nngừa\ntưa\nMưa\nTrưa\nThưa\nrữa\nSữa\nlữa\nxửa\nĐứa\nChữa\nThừa\nRửa\nsứa\nchửa\nnưa\nrựa\nvửa\nNửa\nLửa\nnứa\nbựa\nngữa\nvữa\nnữạ\nxứa\nngưa\nvưa\ntứa\ncữa\nhưa\nừa\nựa\nXưa\nDừa\nửa\nNgựa\nữa\nmừa\nxừa\nxựa\ngiưã\nđữa\ngiưa\nmưạ\nbữa \nhữa\nNhựa\nDứa\nDưa\nSửa\nTựa\nHứa\nNưa\nLứa\nLừa\nmửa\nChứa\nRứa\ncừa\ngiữa s\ngừa\nnhưa\ngiữa t',)

In [20]:
tmp_set = set(rows[0][0].split('\n'))

In [21]:
tmp_set

{'Bữa',
 'Chưa',
 'Chứa',
 'Chữa',
 'Cửa',
 'Dưa',
 'Dứa',
 'Dừa',
 'Dựa',
 'Giữa',
 'Hứa',
 'Lứa',
 'Lừa',
 'Lửa',
 'Mưa',
 'Ngựa',
 'Nhựa',
 'Nưa',
 'Nửa',
 'Rứa',
 'Rửa',
 'Sửa',
 'Sữa',
 'Thưa',
 'Thừa',
 'Trưa',
 'Tựa',
 'Vừa',
 'Xưa',
 'bừa',
 'bửa',
 'bữa',
 'bữa ',
 'bựa',
 'chưa',
 'chứa',
 'chừa',
 'chửa',
 'chữa',
 'cưa',
 'cứa',
 'cừa',
 'cửa',
 'cữa',
 'cựa',
 'dưa',
 'dứa',
 'dừa',
 'dựa',
 'giưa',
 'giưã',
 'giửa',
 'giữa',
 'giữa s',
 'giữa t',
 'gừa',
 'hưa',
 'hứa',
 'hữa',
 'khứa',
 'lưa',
 'lứa',
 'lừa',
 'lửa',
 'lữa',
 'lựa',
 'mưa',
 'mưạ',
 'mừa',
 'mửa',
 'ngưa',
 'ngứa',
 'ngừa',
 'ngửa',
 'ngữa',
 'ngựa',
 'nhưa',
 'nhựa',
 'nưa',
 'nứa',
 'nửa',
 'nữa',
 'nữạ',
 'rứa',
 'rửa',
 'rữa',
 'rựa',
 'sưa',
 'sứa',
 'sửa',
 'sữa',
 'thưa',
 'thừa',
 'thửa',
 'trưa',
 'tưa',
 'tứa',
 'từa',
 'tựa',
 'vưa',
 'vừa',
 'vửa',
 'vữa',
 'vựa',
 'xưa',
 'xưạ',
 'xứa',
 'xừa',
 'xửa',
 'xựa',
 'Đưa',
 'Đứa',
 'đưa',
 'đứa',
 'đữa',
 'ưa',
 'ứa',
 'ừa',
 'ửa',
 'ữa',
 'ựa'}

In [15]:
conn.commit()

# Close the connection
conn.close()

[('mưa', 10762),
 ('giữa', 9982),
 ('chưa', 9227),
 ('xưa', 6424),
 ('lửa', 5161),
 ('vừa', 4846),
 ('nữa', 4404),
 ('đưa', 3792),
 ('cửa', 3648),
 ('nửa', 2784),
 ('đứa', 2519),
 ('ngựa', 1913),
 ('trưa', 1674),
 ('thưa', 1347),
 ('tựa', 1344),
 ('bữa', 1189),
 ('thừa', 809),
 ('sữa', 664),
 ('dừa', 553),
 ('chửa', 546),
 ('hứa', 457),
 ('rửa', 416),
 ('nhựa', 377),
 ('lừa', 374),
 ('ngửa', 369),
 ('ứa', 344),
 ('chứa', 308),
 ('dựa', 295),
 ('dưa', 293),
 ('ưa', 278),
 ('lựa', 244),
 ('sửa', 240),
 ('chừa', 206),
 ('lứa', 186),
 ('rứa', 160),
 ('cựa', 158),
 ('chữa', 152),
 ('nứa', 149),
 ('cứa', 145),
 ('bừa', 118),
 ('thửa', 106),
 ('cưa', 97),
 ('rữa', 96),
 ('dứa', 71),
 ('vữa', 48),
 ('ngứa', 46),
 ('lưa', 43),
 ('mựa', 39),
 ('sưa', 32),
 ('mửa', 26),
 ('bửa', 26),
 ('nưa', 26),
 ('sứa', 25),
 ('rựa', 24),
 ('tứa', 23),
 ('ngừa', 23),
 ('xửa', 23),
 ('lữa', 21),
 ('bưa', 20),
 ('bứa', 11),
 ('tưa', 11),
 ('vựa', 10),
 ('khứa', 9),
 ('gữa', 7),
 ('đừa', 6),
 ('ngữa', 5),
 ('phứa

In [19]:
old_rows = rows

In [11]:
van = "ang e"
cursor.execute(f"SELECT words FROM van_indexing WHERE id = '{van}'")
rows = cursor.fetchall()
rows

[('dáng vẻ\nnàng sẽ\ndáng mẹ\nđang nghe\nhàng tre\nđáng lẽ\nnàng nghe\ncàng sẽ\nđang hé\nhàng me\nvàng nhẹ\ncàng nghe\nnàng khẽ\nsáng sẽ\nđang khẽ\nnàng ghé\nnàng nhẹ\nđang vẽ\nsáng nghe\nnàng thẻ\ntháng mẹ\nđang nhẹ\nchàng sẽ\nmạng che\nnàng che\nmang vẻ\ntháng sẽ\nnàng đè\nsang hè\nchang the\nđang xé\nmang mẹ\nsang ghé\nsang mẹ\nđang se\nsang sẽ\nnàng vẽ\nvàng khẽ\nnàng vẻ\nsáng che\ntang de\nngang sẽ\nlàng xe\ntháng đẻ\nnàng hé\ncàng che\nnàng xé\nđang ghé\ncàng trẻ\nvang khẽ\nđáng nghe\nvang nhẹ\nvàng ghé\nsang ve\nđang bé\nnàng mẹ\nnàng xe\nhàng sẽ\nmang xe\nnàng té\ncàng xe\nvàng vẻ\nmang nhẹ\ndáng vẽ\nngang che\nlàng e\nsang kẻ\nvàng se\nQuang Le\nsang nè\nđang ngh e\nhàng e\ncàng bé\nđang phe\nđang đè\nsáng nhẹ\nlạng lẽ\ncàng bẻ\ngang de\nsáng vẽ\nmang khẽ\nđáng nhẽ\nhạng vé\nvàng xe\nvàng nghe\nsáng mẹ\nmang me\ncàng ghẹ\nnàng nè\ndang the\nmang thẻ\nquang sẽ\nnàng đẻ\ncàng nhẹ\ntràng sẽ\nnàng bẻ\nquang nhẹ\nquang né\nnàng nhe\nmang sẽ\nquang che\nhàng nghe\ngiang nghe\nnàng k

In [12]:
vn_grammar_handler.bo_dau_va_chuyen_van("làng quê")

'lang quê'

In [13]:
vn_grammar_handler.clean_phu_am(vn_grammar_handler.bo_dau_va_chuyen_van("làng quê"))

'ang ê'

In [14]:
vn_grammar_handler.lay_van_cua_tu_hoac_doan(vn_grammar_handler.bo_dau_va_chuyen_van("làng quê"))

'ang ê'

In [15]:
vn_grammar_handler.pattern_remove_phu_am

re.compile(r'((^|\s)ngh(?!\s))|((^|\s)ch(?!\s))|((^|\s)gh(?!\s))|((^|\s)gi(?!\s))|((^|\s)kh(?!\s))|((^|\s)ng(?!\s))|((^|\s)nh(?!\s))|((^|\s)ph(?!\s))|((^|\s)qu(?!\s))|((^|\s)th(?!\s))|((^|\s)tr(?!\s))|((^|\s)b(?!\s))|((^|\s)c(?!\s))|((^|\s)d(?!\s))|((^|\s)g(?!\s))|((^|\s)h(?!\s))|((^|\s)k(?!\s))|((^|\s)l(?!\s))|((^|\s)m(?!\s))|((^|\s)n(?!\s))|((^|\s)p(?!\s))|((^|\s)q(?!\s))|((^|\s)r(?!\s))|((^|\s)s(?!\s))|((^|\s)t(?!\s))|((^|\s)v(?!\s))|((^|\s)x(?!\s))|((^|\s)đ(?!\s))',
           re.UNICODE)

In [8]:
import re

pattern = r"(?:(?<=\s))ngh(?!\s)"
text = "This is an example nghwith ngh at the beginning ngh\tand ngh ngh\nin different situations."

result = re.sub(pattern, "", text)
print(result)

This is an example with ngh at the beginning ngh	and ngh ngh
in different situations.
